<center><h1 style="color:#173F8A;"> Metodos para Ecuaciones Diferenciales, IMT3410, 2024-2 </h1></center>
<h3 style="color:#173F8A;text-align:right;"> Instituto de Ingenieria Matematica y Computacional<br>  Pontificia Universidad Catolica de Chile <br>  </h3>
<h3 style="color:#0176DE;text-align:right;"> Profesor. Manuel A. Sanchez<br> </h3>
<hr style="border:2px solid #03122E"> </hr>

<!-- Palette colors UC: celeste:#0176DE, azul #173F8A, azul oscuro: #03122E, amarillo: #FEC60D, amarillo oscuro: #E3AE00 -->
<!--
<figure>
<img align ="right" src="IMClogo.png" alt="logo" width="250" height="400"><br><br><br><br><br>
</figure>
 -->

<h2 style="color:#03122E;text-align:center;"> Capitulo 2. Metodos para Ecuaciones Diferenciales Parciales Elipticas<br> </h2>
<h3 style="color:#03122E;text-align:center;">             Discontinuous Galerkin Methods (DG Methods) <br> </h3>
<h3 style="color:#03122E;text-align:center;">             Ecuacion de Poisson <br> </h3>
<hr style="border:3px solid #E3AE00 "> </hr>

In [90]:
# NGSolve Libraries
from netgen.geom2d import unit_square
from ngsolve import *
from ngsolve.webgui import Draw
#import netgen.gui

# Domain geometry
geo = unit_square
# Generate mesh with meshsize
hh = .1
mesh = Mesh(geo.GenerateMesh(maxh=hh))

# Manufactured solution
uexact = CoefficientFunction( 16*x*(1-x)*y*(1-y) )
qexact = CoefficientFunction((-16*(1-2*x)*y*(1-y), -16*(1-2*y)*x*(1-x)  ))
# Source according 
f      = 32*y*(1-y)+32*x*(1-x)
Draw(uexact, mesh, 'solucion exacta u ')
Draw(qexact, mesh, 'solucion exacta q')
#print(mesh.GetBoundaries())

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

BaseWebGuiScene

# DG method

Consideremos el problema eliptico
\begin{equation}
\begin{array}{rclll}
\displaystyle
-\Delta u & = & f & \mbox{en} & \Omega \\
\displaystyle u & = & 0 & \mbox{sobre} & \partial \Omega 
\end{array}
\end{equation}

Reescribimos la ecuacion diferencial como un sistema de primer orden introduciendo el flujo $q$, y $a = c^{-1}$
\begin{equation}
\begin{array}{rclll}
\displaystyle  \sigma & = & \nabla u & \mbox{en} & \Omega \\
\displaystyle
-\nabla\cdot( \sigma ) & = & f & \mbox{en} & \Omega \\
\displaystyle u & = & 0 & \mbox{sobre} & \partial \Omega 
\end{array}
\end{equation}


Entonces la formulacion variacional mixta (busca dos soluciones) queda: Hallar $(\sigma_h,u_h)\in \Sigma_h\times W_h$

\begin{equation}
\begin{array}{r}
& \displaystyle \int_{\Omega} \sigma_h \cdot \tau\,dx = - \int_{\Omega} u_h\,\nabla_h\cdot \tau\,dx + \sum_{K\in\mathcal T_h}\int_{\partial K} \phi_{u} \tau\cdot n\,ds \\
&\displaystyle \int_{\Omega} \sigma_h \cdot \nabla_h v\,dx - \sum_{K\in \mathcal T_h}\int_{\partial K} v \phi_{\sigma} \cdot n \,ds = 
 \int_{\Omega} f v\,dx
\end{array}
\end{equation}

Espacios de Galerkin discontinuo:
\begin{align*}
\Sigma_h & = \{r\in L^{1}(\Omega)^{d}: (\tau_i)|_{K}\in \mathcal P_{k},\,\forall K\in \mathcal T_h,\,i=1,...,d\},\\ 
W_h &= \{v\in L^{1}(\Omega): v|_{K}\in \mathcal P_{k},\,\forall K\in \mathcal T_h\}
\end{align*}

# Formulacion Mixta Local Discontinuous Galerkin

\begin{equation}
\phi_{u}(u_h) = \begin{cases}
\{u_h\} - \beta\cdot [u_h],&\quad\text{sobre } \mathcal F_h^{i}, \\
0 ,&\quad\text{sobre } \mathcal F_h^{\partial}, 
\end{cases}
\end{equation}
\begin{equation}
\phi_{\sigma}(u_h,\sigma_h) = \begin{cases}
\{\sigma_h\} + \beta [\sigma_h] - \eta_{F} h_{F}^{-1}[u_h],&\quad\text{sobre } \mathcal F_h^{i}, \\
\sigma_h - \eta_F h_F^{-1} u_h n ,&\quad\text{sobre } \mathcal F_h^{\partial}, 
\end{cases}
\end{equation}


In [91]:
# Domain geometry
geo = unit_square
# Generate mesh with meshsize
hh = .25
mesh = Mesh(geo.GenerateMesh(maxh=hh))
# Mixed form
order = 1
Wh = L2(mesh, order=order, dgjumps=True)
Vh = VectorL2(mesh, order=order, dgjumps=True)
fes = FESpace([Vh,Wh] )

# Trial and test functions
sigma, u = fes.TrialFunction()
tau, v = fes.TestFunction()

# Numerical fluxes
n = specialcf.normal(2)
jump_u = (u-u.Other())*n
jump_v = (v-v.Other())*n
jump_s = (sigma-sigma.Other())*n
jump_t = (tau-tau.Other())*n

mean_u = 0.5*(u+u.Other())
mean_v = 0.5*(v+v.Other())
mean_s = 0.5*(sigma+sigma.Other())
mean_t = 0.5*(tau+tau.Other())

eta = max(4*order**2, 4)
beta = CoefficientFunction((1.0,1.0))
h = specialcf.mesh_size
# LDG fluxes
phi_u_i = mean_u - beta*jump_u
phi_s_i = mean_s + beta*jump_s - eta/h*jump_u 
phi_u_b = 0
phi_s_b = sigma - eta/h*u*n 

dS = dx(element_boundary=True)
dFhi = dx(skeleton=True)
dFhb = ds(skeleton=True)

a_dg = BilinearForm(fes)
# first equation
a_dg += (sigma*tau + u*div(tau) )* dx -phi_u_i*(jump_t)*dFhi- phi_u_b*(tau*n)*dFhb 
# second equation
a_dg += (sigma*grad(v))* dx - (phi_s_i*jump_v)*dFhi - (phi_s_b*v*n)*dFhb
a_dg.Assemble()

fdg = LinearForm(fes)
fdg += f*v*dx 
fdg.Assemble()

gf = GridFunction(fes)
gfq, gfu = gf.components
gf.vec.data = a_dg.mat.Inverse() * fdg.vec
Draw (gfu)
Draw (gfq)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

BaseWebGuiScene

## Nonzero Dirichlet boundary conditions and Neumann conditions

Consideremos el problema eliptico
\begin{equation}
\begin{array}{rclll}
\displaystyle
-\Delta u & = & f & \mbox{en} & \Omega \\
\displaystyle u & = & u_{D} & \mbox{sobre} & \Gamma_{D} \\ 
\displaystyle \nabla u\cdot n & = & g_{N} & \mbox{sobre} & \Gamma_{N}
\end{array}
\end{equation}

Reescribimos la ecuacion diferencial como un sistema de primer orden introduciendo el flujo $\sigma$, 
\begin{equation}
\begin{array}{rclll}
\displaystyle  \sigma & = & \nabla u & \mbox{en} & \Omega \\
\displaystyle
-\nabla\cdot( \sigma ) & = & f & \mbox{en} & \Omega \\
\displaystyle u & = & u_{D}  & \mbox{sobre} & \Gamma_{D} \\
\displaystyle \nabla u \cdot n & = & g_{N}  & \mbox{sobre} & \Gamma_{N}
\end{array}
\end{equation}


Entonces la formulacion variacional mixta (busca dos soluciones) queda: Hallar $(\sigma_h,u_h)\in \Sigma_h\times W_h$

\begin{equation}
\begin{array}{r}
& \displaystyle \int_{\Omega} \sigma_h \cdot \tau\,dx = - \int_{\Omega} u_h\,\nabla_h\cdot \tau\,dx + \sum_{K\in\mathcal T_h}\int_{\partial K} \phi_{u} \tau\cdot n\,ds \\
&\displaystyle \int_{\Omega} \sigma_h \cdot \nabla_h v\,dx - \sum_{K\in \mathcal T_h}\int_{\partial K} v \phi_{\sigma} \cdot n \,ds = 
 \int_{\Omega} f v\,dx
\end{array}
\end{equation}

Espacios de Galerkin discontinuo:
\begin{align*}
\Sigma_h & = \{r\in L^{1}(\Omega)^{d}: (\tau_i)|_{K}\in \mathcal P_{k},\,\forall K\in \mathcal T_h,\,i=1,...,d\},\\ 
W_h &= \{v\in L^{1}(\Omega): v|_{K}\in \mathcal P_{k},\,\forall K\in \mathcal T_h\}
\end{align*}

# Formulacion Mixta Local Discontinuous Galerkin

\begin{equation}
\phi_{u}(u_h) = \begin{cases}
\{u_h\} - \beta\cdot [u_h],&\quad\text{sobre } \mathcal F_h^{i}, \\
0 ,&\quad\text{sobre } \mathcal F_h^{\Gamma_{D}},  \\
u ,&\quad\text{sobre } \mathcal F_h^{\Gamma_{N}}, 
\end{cases}
\end{equation}
\begin{equation}
\phi_{\sigma}(u_h,\sigma_h) = \begin{cases}
\{\sigma_h\} + \beta [\sigma_h] - \eta_{F} h_{F}^{-1}[u_h],&\quad\text{sobre } \mathcal F_h^{i}, \\
\sigma_h - \eta_F h_F^{-1} (u_h n-u_D n) ,&\quad\text{sobre } \mathcal F_h^{\Gamma_{D}}, \\
g_{N}n ,&\quad\text{sobre } \mathcal F_h^{\Gamma_{N}},
\end{cases}
\end{equation}


In [92]:
# NGSolve Libraries
from netgen.geom2d import unit_square
from ngsolve import *
from ngsolve.webgui import Draw
#import netgen.gui

# Domain geometry
geo = unit_square
# Generate mesh with meshsize
hh = .1
mesh = Mesh(geo.GenerateMesh(maxh=hh))

# Manufactured solution
uexact = CoefficientFunction( 16*x*(1-x)*y*(1-y) +y )
qexact = CoefficientFunction((-16*(1-2*x)*y*(1-y), -16*(1-2*y)*x*(1-x) +1  ))

uD = y
gN = 16*(1-2*1.0)*x*(1-x) + 1
# Source according 
f      = 32*y*(1-y)+32*x*(1-x)
Draw(uexact, mesh, 'solucion exacta u ')
Draw(qexact, mesh, 'solucion exacta q')
#print(mesh.GetBoundaries())

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

BaseWebGuiScene

In [93]:
# Domain geometry
geo = unit_square
# Generate mesh with meshsize
hh = .05
mesh = Mesh(geo.GenerateMesh(maxh=hh))
# Mixed form
order = 1
Wh = L2(mesh, order=order, dgjumps=True)
Vh = VectorL2(mesh, order=order, dgjumps=True)
fes = FESpace([Vh,Wh] )

# Trial and test functions
sigma, u = fes.TrialFunction()
tau, v = fes.TestFunction()

# Numerical fluxes
n = specialcf.normal(2)
jump_u = (u-u.Other())*n
jump_v = (v-v.Other())*n
jump_s = (sigma-sigma.Other())*n
jump_t = (tau-tau.Other())*n

mean_u = 0.5*(u+u.Other())
mean_v = 0.5*(v+v.Other())
mean_s = 0.5*(sigma+sigma.Other())
mean_t = 0.5*(tau+tau.Other())

eta = max(4*order**2, 4)
beta = CoefficientFunction((1.0,1.0))
h = specialcf.mesh_size

# LDG fluxes
phi_u_i = mean_u - beta*jump_u
phi_s_i = mean_s + beta*jump_s - eta/h*jump_u 
phi_u_bdir = uD
phi_s_bdir = sigma - eta/h*u*n 
phi_u_bneu = u
phi_s_bneu_n = gN 

dS = dx(element_boundary=True)
dFhi = dx(skeleton=True)
dFhbdir = ds(skeleton=True, definedon='left|bottom|right')
dFhbneu = ds(skeleton=True, definedon='top')

fdg = LinearForm(fes)
fdg += f*v*dx 

a_dg = BilinearForm(fes)

# first equation
a_dg += (sigma*tau + u*div(tau) ) * dx 
a_dg += -phi_u_i*(jump_t) * dFhi 
# a_dg += -u*(tau*n) * dFhbdir
a_dg += -phi_u_bneu*(tau*n) * dFhbneu
fdg += uD*(tau*n) * dFhbdir

# second equation
a_dg += (sigma*grad(v)) * dx 
a_dg += -(phi_s_i*jump_v) * dFhi 
a_dg += -(phi_s_bdir*v*n) * dFhbdir
# a_dg += -(phi_s_bneu_n*v) * dFhbneu
fdg += (eta/h*(uD*n )*v*n) * dFhbdir
fdg += (gN*v) * dFhbneu

a_dg.Assemble()
fdg.Assemble()

gf = GridFunction(fes)
gfq, gfu = gf.components
gf.vec.data = a_dg.mat.Inverse() * fdg.vec
Draw (gfu)
Draw (gfq)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

BaseWebGuiScene

## Primal form

In [10]:
order = 1
Wh = L2(mesh, order=order, dgjumps=True)

# Trial and test functions
u = Wh.TrialFunction()
v = Wh.TestFunction()

# Numerical fluxes
n = specialcf.normal(2)
jump_u = (u-u.Other())*n
jump_v = (v-v.Other())*n
jump_gradu = (grad(u)-grad(u.Other()) )*n
jump_gradv = (grad(v)-grad(v.Other()) )*n

mean_u = 0.5*(u+u.Other())
mean_v = 0.5*(v+v.Other())
mean_gradu = 0.5*(grad(u)+grad(u.Other()) )
mean_gradv = 0.5*(grad(v)+grad(v.Other()) )

eta = max(4*order**2, 4)
beta = CoefficientFunction((1.0,1.0))
h = specialcf.mesh_size
# LDG fluxes
phi_u_i = mean_u - beta*jump_u
phi_s_i = mean_gradu + beta*jump_gradu - eta/h*jump_u 
phi_u_b = 0
phi_s_b = grad(u) - eta/h*u*n 

dS = dx(element_boundary=True)
dFhi = dx(skeleton=True)
dFhb = ds(skeleton=True)

a_dg = BilinearForm(Wh)
# first equation
a_dg += InnerProduct(grad(u),grad(v))* dx 
a_dg += -InnerProduct(jump_u,mean_gradv )*dFhi
a_dg += -InnerProduct((u*n),(grad(v)))*dFhb
a_dg += -phi_s_i * jump_v *  dFhi
a_dg += -phi_s_i * (v*n) * dFhb
a_dg += (phi_u_i-mean_u)*jump_gradv*dFhi
a_dg.Assemble()

fdg = LinearForm(Wh)
fdg += f*v*dx 
fdg.Assemble()

gfu = GridFunction(Wh)
gfu.vec.data = a_dg.mat.Inverse() * fdg.vec
Draw (gfu)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

BaseWebGuiScene

# Formulacion primal e Non-symmetric Interior Penalty Method


\begin{equation}
a_{h}(u_h,v) = \int_{\Omega}\nabla_h u_h\cdot\nabla v_h + \int_{\mathcal F_h} \eta_F h_F^{-1}[u_h]\cdot[ v] +[u_h]\cdot\{\nabla_h v\} - \{\nabla_h u_h\}[v]
\end{equation}
para $u_h,v\in W_h = \{v\in L^{1}(\Omega):\,v|_{K}\in \mathcal P_k,\,\forall K\in \mathcal T_h\}$

In [56]:
# NIPG
order = 4
Wh = L2(mesh, order=order, dgjumps=True)
u,v = Wh.TnT()



jumpn_u = n*(u-u.Other())
jumpn_v = n*(v-v.Other())
n = specialcf.normal(2)
mean_du = 0.5 * (grad(u)+grad(u.Other()))
mean_dv = 0.5 * (grad(v)+grad(v.Other()))

# jump_u = u-u.Other()
# jump_v = v-v.Other()
# mean_dudn = 0.5*n * (grad(u)+grad(u.Other()))
# mean_dvdn = 0.5*n * (grad(v)+grad(v.Other()))


etaF = 4*(order+1)**2
alpha = etaF
h = specialcf.mesh_size
ah = BilinearForm(Wh)
ah += grad(u)*grad(v) * dx
ah += etaF/h*jumpn_u*jumpn_v*dx(skeleton=True)
ah += jumpn_u*mean_dv*dx(skeleton=True)
ah += -mean_du*jumpn_v*dx(skeleton=True)
ah += etaF/h*u*n*v*n*ds(skeleton=True)
ah += u*n*grad(v)*ds(skeleton=True)
ah += -grad(u)*v*n*ds(skeleton=True)

ah.Assemble()

lf = LinearForm(f*v*dx).Assemble()

gfu = GridFunction(Wh, name="uDG")
gfu.vec.data = ah.mat.Inverse() * lf.vec
Draw (gfu)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

BaseWebGuiScene

## Advection diffusion equation

$$ \beta \cdot \nabla u - \epsilon \Delta u = 0 $$


Reescribimos la ecuacion diferencial como un sistema de primer orden introduciendo el flujo $\sigma$
\begin{equation}
\begin{array}{rclll}
\displaystyle  \sigma & = & \nabla u & \mbox{en} & \Omega \\
\displaystyle
\beta \cdot \nabla u-\epsilon \nabla\cdot( \sigma ) & = & f & \mbox{en} & \Omega \\
\displaystyle u & = & 0 & \mbox{sobre} & \partial \Omega 
\end{array}
\end{equation}


Entonces la formulacion variacional mixta (busca dos soluciones) queda: Hallar $(\sigma_h,u_h)\in \Sigma_h\times W_h$

\begin{equation}
\begin{array}{r}
& \displaystyle \int_{\Omega} \sigma_h \cdot \tau\,dx = - \int_{\Omega} u_h\,\nabla_h\cdot \tau\,dx + \sum_{K\in\mathcal T_h}\int_{\partial K} \phi_{u} \tau\cdot n\,ds \\
&\displaystyle 
-\int_{\Omega}u \beta\cdot\nabla_h v +
\sum_{K\in \mathcal T_h}\int_{\partial K} \phi_{u} (\beta \cdot n) v \,ds +\int_{\Omega} \sigma_h \cdot \nabla_h v\,dx - \sum_{K\in \mathcal T_h}\int_{\partial K} v \phi_{\sigma} \cdot n \,ds = 
 \int_{\Omega} f v\,dx
\end{array}
\end{equation}

In [83]:
from netgen.geom2d import SplineGeometry
geo = SplineGeometry()

pnts =[(0,0),(5,0),(5,1),(0,1)]
p1,p2,p3,p4 = [geo.AppendPoint(*pnt) for pnt in pnts]
curves = [[["line",p1,p2],"bottom"],
          [["line",p2,p3],"right"],
          [["line",p3,p4],"top"],
          [["line",p4,p1],"left"]]
[geo.Append(c,bc=bc) for c,bc in curves]
ngmesh = geo.GenerateMesh(maxh=0.05)
mesh = Mesh(ngmesh)
print(mesh.GetBoundaries())

('bottom', 'right', 'top', 'left')


In [89]:
f = 0
epsilon= 0.1
# Mixed form
order = 1
Wh = L2(mesh, order=order, dgjumps=True)
Vh = VectorL2(mesh, order=order, dgjumps=True)
fes = FESpace([Vh,Wh] )

# Trial and test functions
sigma, u = fes.TrialFunction()
tau, v = fes.TestFunction()

# Numerical fluxes
n = specialcf.normal(2)
jump_u = (u-u.Other())*n
jump_v = (v-v.Other())*n
jump_s = (sigma-sigma.Other())*n
jump_t = (tau-tau.Other())*n

mean_u = 0.5*(u+u.Other())
mean_v = 0.5*(v+v.Other())
mean_s = 0.5*(sigma+sigma.Other())
mean_t = 0.5*(tau+tau.Other())

eta = max(4*order**2, 4)
beta = CoefficientFunction((1.0,1.0))
h = specialcf.mesh_size

# LDG fluxes
phi_u_i = mean_u - beta*jump_u
phi_s_i = mean_s + beta*jump_s - eta/h*jump_u 
phi_u_bdir0 = 0
phi_u_bdir1 = 1.0
phi_s_bdir = sigma - eta/h*u*n 
phi_u_bneu = u
phi_s_bneu_n = 0 
gN = 0

dS = dx(element_boundary=True)
dFhi = dx(skeleton=True)
dFhbdir0 = ds(skeleton=True, definedon='left')
dFhbdir1 = ds(skeleton=True, definedon='top|bottom')
dFhbneu = ds(skeleton=True, definedon='right')

fdg = LinearForm(fes)
# fdg += f*v*dx 

a_dg = BilinearForm(fes)

# first equation
a_diff = (sigma*tau + u*div(tau) ) * dx 
a_diff += -phi_u_i*(jump_t) * dFhi 
# a_dg += -u*(tau*n) * dFhbdir
a_diff += -phi_u_bneu*(tau*n) * dFhbneu
fdiff = 1.0*(tau*n) * dFhbdir1

# second equation
a_diff += (sigma*grad(v)) * dx 
a_diff += -(phi_s_i*jump_v) * dFhi 
a_diff += -(phi_s_bdir*v*n) * dFhbdir
# a_dg += -(phi_s_bneu_n*v) * dFhbneu
fdiff += (eta/h*(1*n )*v*n) * dFhbdir1
fdiff += (gN*v) * dFhbneu







b = CoefficientFunction((4*y*(1-y), 0))
a_adv = -b * grad(v)*u*dx
a_adv += phi_u_i * (b*n) * v * dFhi
a_adv += (0) * (b*n) * v * dFhb_dir_0
# a_adv += (1.0) * (b*n) * v * dFhb_dir_1# este pasa al lado derecho
a_adv += (phi_u_bneu) * (b*n) * v * dFhb_neu
fadv = -(1.0)*(b*n)*v*dFhb_dir_1

a_dg += epsilon*a_diff + a_adv
fdg += epsilon*fdiff + fadv
a_dg.Assemble()

fdg.Assemble()

gf = GridFunction(fes)
gfq, gfu = gf.components
gf.vec.data = a_dg.mat.Inverse() * fdg.vec
Draw (gfu)
Draw (gfq)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

BaseWebGuiScene